### pVAR models for timely estimates $-$ Example "single-m1" with DE data

The code herein is pulled from the `R` source code [`DE-single-m1.R`](https://github.com/eurostat/panelvar/blob/master/model/DE-single-m1.R) introduced in the [`panelvar`](https://github.com/eurostat/panelvar) project.

This example can be adapted to run any [other model](https://github.com/eurostat/panelvar/blob/master/model) made available throughout the project.

Load functions:

In [1]:
library("zoo")
library("lubridate")
library("forecast")
library("imputeTS")


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘lubridate’


The following object is masked from ‘package:base’:

    date


Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Registered S3 methods overwritten by 'forecast':
  method             from    
  fitted.fracdiff    fracdiff
  residuals.fracdiff fracdiff


Attaching package: ‘imputeTS’


The following object is masked from ‘package:zoo’:

    na.locf




Define working directories. Note that we adapt here the location of `wd1` since: _(i)_ we run this application from the [notebooks/]( https://github.com/eurostat/panelvar/blob/master/notebooks) folder location, _(ii)_ we don't source the file (the command `dirname(sys.frame(1)$ofile)` works only from "inside" the file only). We also create the `wd3` directory in case it does not exist already:

In [2]:
wd1 <- normalizePath("../model") # dirname(sys.frame(1)$ofile) # absolute path to current directory
wd2 <- "../data/input" # relative paths from wd1
wd3 <- "../data/output/"

dir.create(file.path(wd3), showWarnings = FALSE)

User settings. For the quarterly file, you can only use GDP: 1:GDP, 2:UNR, 3:IP, 4:CPI, 5:FF.

In [3]:
itarg <- 3
jtarg <- c(4,5)
hh <- c(1,3,6,12)  # > 0
ibench <- "AR(1)"
hbounds <- FALSE    # if TRUE, then sigmah*sqrt(h) is used in the bounds otherwise sigmah

Load some additional functions:

In [4]:
setwd(wd1)
source("functions.R")

Load data:

In [5]:
setwd(wd2)

ZM <- load.data(as.matrix(read.csv("DE-M.csv", header=FALSE)))
ZD <- load.data(as.matrix(read.csv("DE-D.csv", header=FALSE)))
ZG <- load.data(as.matrix(read.csv("DE-Google.csv", header=FALSE)))

We have different dates for macrofinance, Google and Reuters. Make sure we use the same dates everywhere.

In [6]:
ds <- max(c(min(ZM$dates), min(ZD$dates), min(ZG$dates)))
de <- min(c(max(ZM$dates), max(ZD$dates), max(ZG$dates)))

xselM <- (ZM$dates>=ds) & (ZM$dates<=de)
xselD <- (ZD$dates>=ds) & (ZD$dates<=de)
xselG <- (ZG$dates>=ds) & (ZG$dates<=de)

XM <- ZM$values[xselM,]
XD <- ZD$values[xselD,]
XG <- ZG$values[xselG,]

Run the calculation:

In [7]:
for(jh in 1:NROW(hh))
{
  setwd(wd1)
  
  h <- hh[jh]
  # give name
  give.name <- paste("DE-M-itarg345", itarg, "-h", h, sep="")

  # make sure you start the loop on the same date
  iinfrom <- "2007-01-31"
  ifrom <- which(rownames(XM)==iinfrom)
  
  # prepare matrices for storage
  mnams <- c("Naive", "AR(1)",
             "LR-MF-T1", "LR-G-T1", "LR-MFGR-T1",
             "LR-MF-T2", "LR-G-T2", "LR-MFGR-T2",
             "AR(1)-MF-T1", "AR(1)-G-T1", "AR(1)-MFGR-T1",
             "AR(1)-MF-T2", "AR(1)-G-T2", "AR(1)-MFGR-T2",
             "VAR(1)",
             "VARX(1)-MF-T1", "VARX(1)-G-T1", "VARX(1)-MFGR-T1",
             "VARX(1)-MF-T2", "VARX(1)-G-T2", "VARX(1)-MFGR-T2")
  fest <- matrix(NA, NROW(XM), NROW(mnams))
  rownames(fest) <- rownames(XM)
  colnames(fest) <- mnams
  
  pintv <- seq(0, 1, 0.01)
  fquant <- array(NA, c(NROW(fest), NROW(pintv), NROW(mnams)),
                           dimnames=list(rownames(fest), as.character(pintv), mnams))
  
  for(i in ifrom:(NROW(XM)-h))
  {
    # in-sample
    xm <- XM[1:i, ]
    dm <- as.Date(rownames(xm))
    xd <- make.data.DWtoW(dm, XD)
    xg <- make.data.DWtoW(dm, XG)
    
    # if NAs use interpolation
    xd <- apply(xd, 2, na_interpolation, option="linear")
    xg <- apply(xg, 2, na_interpolation, option="linear")

    # Transform all variables
    xms <- transform.vars(xm, ZM$transf)
    xds <- transform.vars(xd, ZD$transf)
    xgs <- transform.vars(xg, ZG$transf)

    # Extract weekly factors using PCA
    Fw <- na.omit(xds); rnams <- rownames(Fw)
    Fw <- getPCA(Fw); rownames(Fw) <- rnams
    Fg <- na.omit(xgs); rnams <- rownames(Fg)
    Fg <- getPCA(Fg);  rownames(Fg) <- rnams
    
    # Extract target
    y <- as.matrix(xms[,ZM$target==itarg])
    
    # Also extract the other 2 variables for the simple VAR
    x1 <- as.matrix(xms[,ZM$target==jtarg[1]])
    x2 <- as.matrix(xms[,ZM$target==jtarg[2]])
    xx <- cbind(x1, x2)
    colnames(xx) <- paste("X", 1:NCOL(xx), sep="")
    
    # Transform weekly factors to monthly
    # according to the dates
    xF1 <- make.data.WtoM1(as.Date(rownames(y)), as.matrix(Fw[,1]))
    xG1 <- make.data.WtoM1(as.Date(rownames(y)), as.matrix(Fg[,1]))
    xFGR1 <- cbind(xF1, xG1)
    
    xF2 <- make.data.WtoM2(as.Date(rownames(y)), as.matrix(Fw[,1]))
    xG2 <- make.data.WtoM2(as.Date(rownames(y)), as.matrix(Fg[,1]))
    xFGR2 <- cbind(xF2, xG2)
    
    # make a lag
    yL1 <- as.matrix(c(NA, y[1:(NROW(y)-1),]))
    
    # make forecasts
    imodel <- "Naive"; z <- y; source("naive.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "AR(1)"; YY <- y; XX <- lag.uni(as.matrix(y), 1); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "LR-MF-T1"; XX <- xF1; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "LR-G-T1"; XX <- xG1; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "LR-MFGR-T1"; XX <- xFGR1; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "LR-MF-T2"; XX <- xF2; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "LR-G-T2"; XX <- xG2; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]

    imodel <- "LR-MFGR-T2"; XX <- xFGR2; YY <- y; source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "AR(1)-MF-T1"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xF1); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "AR(1)-G-T1"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xG1); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]

    imodel <- "AR(1)-MFGR-T1"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xFGR1); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "AR(1)-MF-T2"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xF2); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "AR(1)-G-T2"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xG2); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]

    imodel <- "AR(1)-MFGR-T2"; YY <- y; XX <- cbind(lag.uni(as.matrix(y), 1), xFGR2); source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VAR(1)"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag))
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-MF-T1"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xF1)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-G-T1"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xG1)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-MFGR-T1"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xFGR1)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-MF-T2"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xF2)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-G-T2"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xG2)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    imodel <- "VARX(1)-MFGR-T2"; glag <- 1
    YY <- y; XX <- cbind(lag.uni(as.matrix(y), glag), lag.multi(xx, glag), xFGR2)
    source("Flinreg.R")
    fest[(i+h), imodel] <- zout[1]; fquant[(i+h), ,imodel] <- zout[2:NROW(zout)]
    
    cat("Now doing ", i, " of ", (NROW(XM)-h), " and h ", h, "\n")
  }
  
  fwhich <- which(ZM$target==itarg)
  ftrue <- as.matrix(XM[,fwhich])
  ftrue <- transform.vars(ftrue, ZM$transf[fwhich])
  
  e <- matrix(ftrue, NROW(ftrue), NCOL(fest))-fest
  rownames(e) <- rownames(fest)
  colnames(e) <- colnames(fest)
  e <- na.omit(e)
  
  ftrue.diff <- transform.vars(as.matrix(ftrue), 3)
  fest.diff <- transform.vars(fest, rep(3, NCOL(fest)))
  ss.diff <- sign(matrix(ftrue.diff, NROW(ftrue.diff), NCOL(fest.diff)))-sign(fest.diff)
  ss.diff <- na.omit(ss.diff)
  
  # Save the output
  setwd(wd3)
  
  ibn <- which(colnames(e)==ibench)
  
  MAE <- colMeans(abs(e))
  RMSFE <- sqrt(colMeans(e^2))
  SSR <- colMeans((ss.diff==0))
  
  stats1 <- cbind(MAE, RMSFE, SSR)
  stats2 <- cbind(MAE/MAE[ibn], RMSFE/RMSFE[ibn], SSR)
  
  DM <- matrix(NA, NROW(stats1), 2)
  for(j in 1:NCOL(e))
  {
    if(j!=ibn){
      DM[j,1] <- dm.test(e[,j], e[,ibn], alternative = c("two.sided"), h=h, power=1)$p.value
      DM[j,2] <- dm.test(e[,j], e[,ibn], alternative = c("two.sided"), h=h, power=2)$p.value
     }
  }
  rownames(stats1) <- rownames(stats2) <- rownames(DM) <- mnams
  colnames(stats1) <- colnames(stats2) <- c("MAE", "RMSFE", "SSR")
  colnames(DM) <- c("DMabs", "DMsq")
  
  write.csv(e, paste(give.name, "-error.csv", sep=""))
  write.csv(stats2, paste(give.name, "-stats2.csv", sep=""))
  write.csv(DM, paste(give.name, "-DMpval.csv", sep=""))
  
  # Print the density plots as well as export the graph data
  
  for(jj in 1:NCOL(fest))
  {
    print.backplots(give.name, ftrue, fest[,jj],
                    fquant[rownames(ftrue), ,mnams[jj]], mnams[jj])
  }
  
  save.image(paste(give.name, ".Rdata", sep=""))
  
}

Now doing  37  of  163  and h  1 
Now doing  38  of  163  and h  1 
Now doing  39  of  163  and h  1 
Now doing  40  of  163  and h  1 
Now doing  41  of  163  and h  1 
Now doing  42  of  163  and h  1 
Now doing  43  of  163  and h  1 
Now doing  44  of  163  and h  1 
Now doing  45  of  163  and h  1 
Now doing  46  of  163  and h  1 
Now doing  47  of  163  and h  1 
Now doing  48  of  163  and h  1 
Now doing  49  of  163  and h  1 
Now doing  50  of  163  and h  1 
Now doing  51  of  163  and h  1 
Now doing  52  of  163  and h  1 
Now doing  53  of  163  and h  1 
Now doing  54  of  163  and h  1 
Now doing  55  of  163  and h  1 
Now doing  56  of  163  and h  1 
Now doing  57  of  163  and h  1 
Now doing  58  of  163  and h  1 
Now doing  59  of  163  and h  1 
Now doing  60  of  163  and h  1 
Now doing  61  of  163  and h  1 
Now doing  62  of  163  and h  1 
Now doing  63  of  163  and h  1 
Now doing  64  of  163  and h  1 
Now doing  65  of  163  and h  1 
Now doing  66 

Check the output files that were created:

In [8]:
ofiles <- list.files(getwd())
ofiles 

[1] "DE-M-itarg3453-h1-AR(1)-G-T1.pdf"      
  [2] "DE-M-itarg3453-h1-AR(1)-G-T2.pdf"      
  [3] "DE-M-itarg3453-h1-AR(1)-MF-T1.pdf"     
  [4] "DE-M-itarg3453-h1-AR(1)-MF-T2.pdf"     
  [5] "DE-M-itarg3453-h1-AR(1)-MFGR-T1.pdf"   
  [6] "DE-M-itarg3453-h1-AR(1)-MFGR-T2.pdf"   
  [7] "DE-M-itarg3453-h1-AR(1).pdf"           
  [8] "DE-M-itarg3453-h1-DMpval.csv"          
  [9] "DE-M-itarg3453-h1-error.csv"           
 [10] "DE-M-itarg3453-h1-LR-G-T1.pdf"         
 [11] "DE-M-itarg3453-h1-LR-G-T2.pdf"         
 [12] "DE-M-itarg3453-h1-LR-MF-T1.pdf"        
 [13] "DE-M-itarg3453-h1-LR-MF-T2.pdf"        
 [14] "DE-M-itarg3453-h1-LR-MFGR-T1.pdf"      
 [15] "DE-M-itarg3453-h1-LR-MFGR-T2.pdf"      
 [16] "DE-M-itarg3453-h1-Naive.pdf"           
 [17] "DE-M-itarg3453-h1-stats2.csv"          
 [18] "DE-M-itarg3453-h1-VAR(1).pdf"          
 [19] "DE-M-itarg3453-h1-VARX(1)-G-T1.pdf"    
 [20] "DE-M-itarg3453-h1-VARX(1)-G-T2.pdf"    
 [21] "DE-M-itarg3453-h1-VARX(1)-MF-T1.pdf"   
 [22] "DE-M-itarg3453-h1-VARX(1)-MF-T2.pdf"   
 [23] "DE-M-itarg3453-h1-VARX(1)-MFGR-T1.pdf" 
 [24] "DE-M-itarg3453-h1-VARX(1)-MFGR-T2.pdf" 
 [25] "DE-M-itarg3453-h1.Rdata"               
 [26] "DE-M-itarg3453-h12-AR(1)-G-T1.pdf"     
 [27] "DE-M-itarg3453-h12-AR(1)-G-T2.pdf"     
 [28] "DE-M-itarg3453-h12-AR(1)-MF-T1.pdf"    
 [29] "DE-M-itarg3453-h12-AR(1)-MF-T2.pdf"    
 [30] "DE-M-itarg3453-h12-AR(1)-MFGR-T1.pdf"  
 [31] "DE-M-itarg3453-h12-AR(1)-MFGR-T2.pdf"  
 [32] "DE-M-itarg3453-h12-AR(1).pdf"          
 [33] "DE-M-itarg3453-h12-DMpval.csv"         
 [34] "DE-M-itarg3453-h12-error.csv"          
 [35] "DE-M-itarg3453-h12-LR-G-T1.pdf"        
 [36] "DE-M-itarg3453-h12-LR-G-T2.pdf"        
 [37] "DE-M-itarg3453-h12-LR-MF-T1.pdf"       
 [38] "DE-M-itarg3453-h12-LR-MF-T2.pdf"       
 [39] "DE-M-itarg3453-h12-LR-MFGR-T1.pdf"     
 [40] "DE-M-itarg3453-h12-LR-MFGR-T2.pdf"     
 [41] "DE-M-itarg3453-h12-Naive.pdf"          
 [42] "DE-M-itarg3453-h12-stats2.csv"         
 [43] "DE-M-itarg3453-h12-VAR(1).pdf"         
 [44] "DE-M-itarg3453-h12-VARX(1)-G-T1.pdf"   
 [45] "DE-M-itarg3453-h12-VARX(1)-G-T2.pdf"   
 [46] "DE-M-itarg3453-h12-VARX(1)-MF-T1.pdf"  
 [47] "DE-M-itarg3453-h12-VARX(1)-MF-T2.pdf"  
 [48] "DE-M-itarg3453-h12-VARX(1)-MFGR-T1.pdf"
 [49] "DE-M-itarg3453-h12-VARX(1)-MFGR-T2.pdf"
 [50] "DE-M-itarg3453-h12.Rdata"              
 [51] "DE-M-itarg3453-h3-AR(1)-G-T1.pdf"      
 [52] "DE-M-itarg3453-h3-AR(1)-G-T2.pdf"      
 [53] "DE-M-itarg3453-h3-AR(1)-MF-T1.pdf"     
 [54] "DE-M-itarg3453-h3-AR(1)-MF-T2.pdf"     
 [55] "DE-M-itarg3453-h3-AR(1)-MFGR-T1.pdf"   
 [56] "DE-M-itarg3453-h3-AR(1)-MFGR-T2.pdf"   
 [57] "DE-M-itarg3453-h3-AR(1).pdf"           
 [58] "DE-M-itarg3453-h3-DMpval.csv"          
 [59] "DE-M-itarg3453-h3-error.csv"           
 [60] "DE-M-itarg3453-h3-LR-G-T1.pdf"         
 [61] "DE-M-itarg3453-h3-LR-G-T2.pdf"         
 [62] "DE-M-itarg3453-h3-LR-MF-T1.pdf"        
 [63] "DE-M-itarg3453-h3-LR-MF-T2.pdf"        
 [64] "DE-M-itarg3453-h3-LR-MFGR-T1.pdf"      
 [65] "DE-M-itarg3453-h3-LR-MFGR-T2.pdf"      
 [66] "DE-M-itarg3453-h3-Naive.pdf"           
 [67] "DE-M-itarg3453-h3-stats2.csv"          
 [68] "DE-M-itarg3453-h3-VAR(1).pdf"          
 [69] "DE-M-itarg3453-h3-VARX(1)-G-T1.pdf"    
 [70] "DE-M-itarg3453-h3-VARX(1)-G-T2.pdf"    
 [71] "DE-M-itarg3453-h3-VARX(1)-MF-T1.pdf"   
 [72] "DE-M-itarg3453-h3-VARX(1)-MF-T2.pdf"   
 [73] "DE-M-itarg3453-h3-VARX(1)-MFGR-T1.pdf" 
 [74] "DE-M-itarg3453-h3-VARX(1)-MFGR-T2.pdf" 
 [75] "DE-M-itarg3453-h3.Rdata"               
 [76] "DE-M-itarg3453-h6-AR(1)-G-T1.pdf"      
 [77] "DE-M-itarg3453-h6-AR(1)-G-T2.pdf"      
 [78] "DE-M-itarg3453-h6-AR(1)-MF-T1.pdf"     
 [79] "DE-M-itarg3453-h6-AR(1)-MF-T2.pdf"     
 [80] "DE-M-itarg3453-h6-AR(1)-MFGR-T1.pdf"   
 [81] "DE-M-itarg3453-h6-AR(1)-MFGR-T2.pdf"   
 [82] "DE-M-itarg3453-h6-AR(1).pdf"           
 [83] "DE-M-itarg3453-h6-DMpval.csv"          
 [84] "DE-M-itarg3453-h6-error.csv"           
 [85] "DE-M-itarg3453-h6-LR-G-T1.pdf"         
 [86] "